In [ ]:
!pip install pyarrow duckdb pandas numpy polars -q

# 📦 Instalação de Pacotes

Antes de começar, vamos instalar os pacotes necessários:

# Capitulo 08 Integracao Pandas Polars

Notebook gerado automaticamente a partir do código fonte python.


## 📚 Importação de Bibliotecas

Importando as bibliotecas necessárias para o capítulo:

In [ ]:
print(f"\n--- {'Pipelines híbridos'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Pipelines híbridos
# Dica: Use duckdb.query() ou funcoes do pyarrow


print("\n" + "="*60)
print(f"Fim do Capítulo 08")
print("="*60)

## 5️⃣ Pipelines Híbridos

Criando pipelines combinando diferentes bibliotecas:

In [ ]:
print(f"\n--- {'Otimizações de tipos'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Otimizações de tipos
# Dica: Use duckdb.query() ou funcoes do pyarrow

## 4️⃣ Otimizações de Tipos

Otimizando tipos de dados para melhor performance:

In [ ]:
print(f"\n--- {'Interoperabilidade DuckDB'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Interoperabilidade DuckDB
# Dica: Use duckdb.query() ou funcoes do pyarrow

## 3️⃣ Interoperabilidade DuckDB

Integração entre DuckDB, Pandas, Polars e Arrow:

In [ ]:
print(f"\n--- {'Conversões Polars ↔ Arrow'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Conversões Polars ↔ Arrow
# Dica: Use duckdb.query() ou funcoes do pyarrow

## 2️⃣ Conversões Polars ↔ Arrow

Convertendo dados entre Polars e Apache Arrow:

In [ ]:
print(f"\n--- {'Conversões Pandas ↔ Arrow'.upper()} ---")

# TODO: Implementar exemplos práticos sobre Conversões Pandas ↔ Arrow
# Dica: Use duckdb.query() ou funcoes do pyarrow

## 1️⃣ Conversões Pandas ↔ Arrow

Convertendo dados entre Pandas e Apache Arrow:

In [ ]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()

## 🔧 Configuração Inicial

Preparando dados de exemplo e conexão com DuckDB: